In [3]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [5]:
subject = 'Mato Grosso Do Sul - Consumo de Cimento (t)'
split_index = 191 #Referente aos 230 anos de input  
train_split = split_index + 1 -30

In [6]:
data = pd.read_csv('2003_mo_model_input_MS.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Mato Grosso do Sul - IDH,Mato Grosso Do Sul - Produção de Cimento (t),Mato Grosso do Sul - Desemprego,Mato Grosso do Sul - value,Mato Grosso do Sul - PIB - Estadual,Mato Grosso do Sul - PIB - Construção Civil,Mato Grosso do Sul - PIB - Per Capita,Mato Grosso do Sul - PIB - Preços de Mercado,Mato Grosso Do Sul - Consumo de Cimento (t)
0,2003-1,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,0.741626,53.166560,8.248824,0.282837,4.562945e+07,2.151917e+06,16.203179,3.957545e+07,33.744
1,2003-2,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,0.741710,53.245714,8.242935,0.284487,4.568706e+07,2.153542e+06,16.206131,3.959155e+07,32.230
2,2003-3,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,0.741794,52.994640,8.237046,0.285379,4.574467e+07,2.155168e+06,16.209084,3.960765e+07,33.559
3,2003-4,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,0.741878,53.019283,8.231157,0.286089,4.580228e+07,2.156794e+06,16.212036,3.962375e+07,34.283
4,2003-5,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,0.741962,53.539593,8.225268,0.286122,4.585990e+07,2.158419e+06,16.214989,3.963985e+07,36.066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72.097125,NaN,0.523445,NaN,NaN,NaN,NaN,103.854
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71.657488,NaN,0.521590,NaN,NaN,NaN,NaN,95.890
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71.966263,NaN,0.518616,NaN,NaN,NaN,NaN,89.679
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72.205635,NaN,0.515040,NaN,NaN,NaN,NaN,99.356


In [7]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Mato Grosso do Sul - IDH,Mato Grosso Do Sul - Produção de Cimento (t),Mato Grosso do Sul - Desemprego,Mato Grosso do Sul - value,Mato Grosso do Sul - PIB - Estadual,Mato Grosso do Sul - PIB - Construção Civil,Mato Grosso do Sul - PIB - Per Capita,Mato Grosso do Sul - PIB - Preços de Mercado
0,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.739598,-1.432541,1.216541,-0.459341,-1.637645,-2.402906,-1.784438,-1.738619
1,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.720624,-1.423955,1.206220,-0.422027,-1.622679,-2.361122,-1.774070,-1.724323
2,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.701651,-1.451188,1.195898,-0.401858,-1.607713,-2.319339,-1.763703,-1.710027
3,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.682677,-1.448515,1.185577,-0.385815,-1.592747,-2.277555,-1.753336,-1.695732
4,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.663704,-1.392079,1.175256,-0.385067,-1.577781,-2.235771,-1.742968,-1.681436
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.190288,-0.831849,-1.205180,0.316446,1.171465,-0.292680,0.923349,1.021439
188,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.165389,-0.799920,-1.216425,0.383902,1.161163,-0.292842,0.905256,1.007593
189,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.140489,-0.711338,-1.227670,0.452548,1.150861,-0.293004,0.887162,0.993747
190,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.115590,-0.636591,-1.238915,0.528940,1.140559,-0.293166,0.869069,0.979902


In [8]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      36.256
1      33.675
2      43.734
3      36.187
4      33.413
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Mato Grosso Do Sul - Consumo de Cimento (t), Length: 240, dtype: float64

In [9]:
# input para treinamento
train_input = input_data.iloc[:train_split]
train_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Mato Grosso do Sul - IDH,Mato Grosso Do Sul - Produção de Cimento (t),Mato Grosso do Sul - Desemprego,Mato Grosso do Sul - value,Mato Grosso do Sul - PIB - Estadual,Mato Grosso do Sul - PIB - Construção Civil,Mato Grosso do Sul - PIB - Per Capita,Mato Grosso do Sul - PIB - Preços de Mercado
0,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.739598,-1.432541,1.216541,-0.459341,-1.637645,-2.402906,-1.784438,-1.738619
1,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.720624,-1.423955,1.206220,-0.422027,-1.622679,-2.361122,-1.774070,-1.724323
2,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.701651,-1.451188,1.195898,-0.401858,-1.607713,-2.319339,-1.763703,-1.710027
3,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.682677,-1.448515,1.185577,-0.385815,-1.592747,-2.277555,-1.753336,-1.695732
4,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.663704,-1.392079,1.175256,-0.385067,-1.577781,-2.235771,-1.742968,-1.681436
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,1.523477,-0.303048,-0.435460,0.586449,1.207622,0.129567,1.062247,1.123802
158,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,1.520281,-0.384401,-0.482211,0.549251,1.211468,0.098041,1.062771,1.125391
159,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,1.517085,-0.431034,-0.528962,0.512196,1.215313,0.066515,1.063296,1.126980
160,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,1.513890,-0.493901,-0.575713,0.470611,1.219159,0.034989,1.063820,1.128569


In [10]:
# Alvo para treinamento
train_target = target_data.iloc[:train_split]
train_target

0      36.256
1      33.675
2      43.734
3      36.187
4      33.413
        ...  
157    66.462
158    79.514
159    63.281
160    77.205
161    71.168
Name: Mato Grosso Do Sul - Consumo de Cimento (t), Length: 162, dtype: float64

In [11]:
#input de test (Ano 2021)
test_input = input_data.iloc[train_split:]
test_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Mato Grosso do Sul - IDH,Mato Grosso Do Sul - Produção de Cimento (t),Mato Grosso do Sul - Desemprego,Mato Grosso do Sul - value,Mato Grosso do Sul - PIB - Estadual,Mato Grosso do Sul - PIB - Construção Civil,Mato Grosso do Sul - PIB - Per Capita,Mato Grosso do Sul - PIB - Preços de Mercado
162,-0.601510,-0.653307,0.157879,-1.255326,1.088045,0.763466,-1.213929,1.507498,-0.558761,-0.669216,0.395858,1.226851,-0.028063,1.064869,1.131747
163,-0.786068,-0.653814,0.548855,-1.118679,1.099377,0.752299,-1.292173,1.504302,-0.590305,-0.715967,0.361218,1.230696,-0.059589,1.065393,1.133336
164,-0.830387,-0.645656,0.385685,-0.936336,1.110709,0.741131,-1.324219,1.501107,-0.641603,-0.762718,0.329709,1.234542,-0.091115,1.065917,1.134925
165,-0.801089,-0.634274,0.420965,-0.931790,1.122042,0.729964,-1.344446,1.497911,-0.706731,-0.809470,0.303630,1.238388,-0.122641,1.066442,1.136514
166,-0.959917,-0.624035,0.548870,-1.168522,1.133374,0.718796,-1.381638,1.494715,-0.754771,-0.856221,0.282160,1.242233,-0.154167,1.066966,1.138104
167,-1.022309,-0.619791,0.222499,-1.285611,1.144706,0.707629,-1.411208,1.491520,-0.803758,-0.902972,0.262035,1.246079,-0.185693,1.067490,1.139693
168,-1.074401,-0.620112,-0.614754,-1.446316,1.156038,0.696461,-1.412953,1.488324,-0.858755,-0.949723,0.247167,1.249925,-0.217219,1.068015,1.141282
169,-1.119597,-0.615973,-0.478229,-1.357832,1.167287,0.681823,-1.491464,1.478012,-0.900149,-0.964452,0.238710,1.249396,-0.223413,1.066514,1.139372
170,-1.078648,-0.609777,-0.393157,-1.235682,1.178536,0.667184,-1.573805,1.467701,-0.923483,-0.979180,0.230368,1.248867,-0.229608,1.065013,1.137461
171,-1.055426,-0.602141,-0.557552,-1.065340,1.189784,0.652545,-1.564950,1.457389,-0.970684,-0.993908,0.219296,1.248338,-0.235802,1.063512,1.135551


In [12]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[train_split:split_index + 1]
test_target

162    80.517
163    81.348
164    76.509
165    71.851
166    71.187
167    58.020
168    63.194
169    60.580
170    69.448
171    74.472
172    63.776
173    81.732
174    83.858
175    87.863
176    69.723
177    72.765
178    77.937
179    64.332
180    77.048
181    65.660
182    65.445
183    78.095
184    80.998
185    69.415
186    87.346
187    82.888
188    79.287
189    81.855
190    77.495
191    56.612
Name: Mato Grosso Do Sul - Consumo de Cimento (t), dtype: float64

In [13]:
def validation_splitter(df, div_factor):
    split_factor = len(df)//div_factor
    positions_to_drop = []
    for i in range(split_factor//2):
        pos = df.shape[0] - (i*div_factor + 1)
        positions_to_drop.append(pos)
        pos = df.shape[0] - (i*div_factor + 3)
        positions_to_drop.append(pos)
    
    return df.drop(positions_to_drop), df.iloc[positions_to_drop]

In [14]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    train, train_val = validation_splitter(train_input, 6)
    target,target_val = validation_splitter(train_target, 6)
#     display(train)
#     display(train_val)
#     display(target)
#     display(target_val)
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val, target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [17]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(25)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [18]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[1246087175, 2184783715, 2582528527, 1053111234, 348725400, 1291293116, 801014609, 3005362731, 3647643016, 2841889144, 2144919270, 4154215439, 2891408204, 3105058538, 1877066431, 238022320, 2521715964, 2330700615, 2597399158, 1262788948, 1097606053, 3636711877, 3003188941, 377532714, 3505716841]


Step: 0 ___________________________________________
val_loss: 62.81085205078125
winner_seed: 1246087175


Step: 1 ___________________________________________
val_loss: 37.40751266479492
winner_seed: 2184783715


Step: 2 ___________________________________________
val_loss: 28.450897216796875
winner_seed: 2582528527


Step: 3 ___________________________________________
val_loss: 44.097755432128906


Step: 4 ___________________________________________
val_loss: 40.52454376220703


Step: 5 ___________________________________________
val_loss: 46.374942779541016


Step: 6 ___________________________________________
val_loss: 38.58902359008789


Step: 7 ___________________________________________


2023-10-14 17:08:34.302685: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 109.46446990966797


Step: 24 ___________________________________________
val_loss: 36.738101959228516


final_seed: 2582528527


In [19]:
trained_model, history = neural_network_model(train_input, 
                                              train_target, 
                                              want_verbose=1, 
                                              seed=winner_seed)

Epoch 1/10000
5/5 [==============================] - 1s 27ms/step - loss: 5324.3745 - val_loss: 25440.9980
Epoch 2/10000
5/5 [==============================] - 0s 6ms/step - loss: 6052.7866 - val_loss: 15549.3447
Epoch 3/10000
5/5 [==============================] - 0s 7ms/step - loss: 4801.5972 - val_loss: 7877.1777
Epoch 4/10000
5/5 [==============================] - 0s 8ms/step - loss: 4121.6235 - val_loss: 6036.8262
Epoch 5/10000
5/5 [==============================] - 0s 7ms/step - loss: 3688.6619 - val_loss: 2726.4482
Epoch 6/10000
5/5 [==============================] - 0s 6ms/step - loss: 2124.7351 - val_loss: 5712.1484
Epoch 7/10000
5/5 [==============================] - 0s 6ms/step - loss: 1824.8846 - val_loss: 2203.9241
Epoch 8/10000
5/5 [==============================] - 0s 7ms/step - loss: 631.4060 - val_loss: 835.1866
Epoch 9/10000
5/5 [==============================] - 0s 6ms/step - loss: 284.5677 - val_loss: 533.0089
Epoch 10/10000
5/5 [==============================] - 0s

Epoch 80/10000
5/5 [==============================] - 0s 7ms/step - loss: 117.0551 - val_loss: 312.2980
Epoch 81/10000
5/5 [==============================] - 0s 7ms/step - loss: 118.0498 - val_loss: 146.6781
Epoch 82/10000
5/5 [==============================] - 0s 7ms/step - loss: 77.2554 - val_loss: 117.8890
Epoch 83/10000
5/5 [==============================] - 0s 7ms/step - loss: 92.2545 - val_loss: 214.2281
Epoch 84/10000
5/5 [==============================] - 0s 7ms/step - loss: 74.4067 - val_loss: 121.1719
Epoch 85/10000
5/5 [==============================] - 0s 8ms/step - loss: 82.8026 - val_loss: 225.6489
Epoch 86/10000
5/5 [==============================] - 0s 6ms/step - loss: 100.8658 - val_loss: 368.0295
Epoch 87/10000
5/5 [==============================] - 0s 6ms/step - loss: 81.2502 - val_loss: 264.4512
Epoch 88/10000
5/5 [==============================] - 0s 6ms/step - loss: 85.7175 - val_loss: 761.4329
Epoch 89/10000
5/5 [==============================] - 0s 7ms/step - lo

5/5 [==============================] - 0s 6ms/step - loss: 126.9817 - val_loss: 238.5633
Epoch 159/10000
5/5 [==============================] - 0s 6ms/step - loss: 79.0964 - val_loss: 210.0903
Epoch 160/10000
5/5 [==============================] - 0s 7ms/step - loss: 82.7859 - val_loss: 317.7822
Epoch 161/10000
5/5 [==============================] - 0s 7ms/step - loss: 118.7606 - val_loss: 211.0091
Epoch 162/10000
5/5 [==============================] - 0s 7ms/step - loss: 104.2638 - val_loss: 569.3248
Epoch 163/10000
5/5 [==============================] - 0s 7ms/step - loss: 150.9623 - val_loss: 433.3993
Epoch 164/10000
5/5 [==============================] - 0s 7ms/step - loss: 115.1594 - val_loss: 331.1935
Epoch 165/10000
5/5 [==============================] - 0s 6ms/step - loss: 81.3850 - val_loss: 175.0906
Epoch 166/10000
5/5 [==============================] - 0s 6ms/step - loss: 76.4923 - val_loss: 180.8896
Epoch 167/10000
5/5 [==============================] - 0s 7ms/step - loss: 

5/5 [==============================] - 0s 6ms/step - loss: 156.1551 - val_loss: 435.8315
Epoch 237/10000
5/5 [==============================] - 0s 6ms/step - loss: 164.7958 - val_loss: 346.2262
Epoch 238/10000
5/5 [==============================] - 0s 6ms/step - loss: 139.8496 - val_loss: 245.0413
Epoch 239/10000
5/5 [==============================] - 0s 7ms/step - loss: 108.7612 - val_loss: 245.1460
Epoch 240/10000
5/5 [==============================] - 0s 7ms/step - loss: 158.0589 - val_loss: 312.0605
Epoch 241/10000
5/5 [==============================] - 0s 7ms/step - loss: 136.8740 - val_loss: 222.7310
Epoch 242/10000
5/5 [==============================] - 0s 6ms/step - loss: 110.8202 - val_loss: 269.0571
Epoch 243/10000
5/5 [==============================] - 0s 6ms/step - loss: 126.4396 - val_loss: 239.1881
Epoch 244/10000
5/5 [==============================] - 0s 7ms/step - loss: 96.8684 - val_loss: 218.1833
Epoch 245/10000
5/5 [==============================] - 0s 6ms/step - los

5/5 [==============================] - 0s 6ms/step - loss: 58.8698 - val_loss: 95.7948
Epoch 315/10000
5/5 [==============================] - 0s 6ms/step - loss: 64.3092 - val_loss: 89.1871
Epoch 316/10000
5/5 [==============================] - 0s 6ms/step - loss: 58.2096 - val_loss: 84.0837
Epoch 317/10000
5/5 [==============================] - 0s 7ms/step - loss: 61.5156 - val_loss: 91.9113
Epoch 318/10000
5/5 [==============================] - 0s 8ms/step - loss: 58.2803 - val_loss: 101.2591
Epoch 319/10000
5/5 [==============================] - 0s 7ms/step - loss: 65.8672 - val_loss: 95.4821
Epoch 320/10000
5/5 [==============================] - 0s 8ms/step - loss: 59.0806 - val_loss: 112.8269
Epoch 321/10000
5/5 [==============================] - 0s 6ms/step - loss: 56.5869 - val_loss: 183.1279
Epoch 322/10000
5/5 [==============================] - 0s 6ms/step - loss: 85.8342 - val_loss: 121.2351
Epoch 323/10000
5/5 [==============================] - 0s 6ms/step - loss: 70.8288 - 

...

5/5 [==============================] - 0s 5ms/step - loss: 41.0395 - val_loss: 108.1693
Epoch 866/10000
5/5 [==============================] - 0s 5ms/step - loss: 48.2028 - val_loss: 258.8541
Epoch 867/10000
5/5 [==============================] - 0s 5ms/step - loss: 65.4783 - val_loss: 115.1413
Epoch 868/10000
5/5 [==============================] - 0s 5ms/step - loss: 50.3035 - val_loss: 112.6506
Epoch 869/10000
5/5 [==============================] - 0s 5ms/step - loss: 52.2432 - val_loss: 96.9772
Epoch 870/10000
5/5 [==============================] - 0s 7ms/step - loss: 48.0455 - val_loss: 142.2740
Epoch 871/10000
5/5 [==============================] - 0s 6ms/step - loss: 77.8831 - val_loss: 74.4330
Epoch 872/10000
5/5 [==============================] - 0s 6ms/step - loss: 82.8872 - val_loss: 71.2046
Epoch 873/10000
5/5 [==============================] - 0s 7ms/step - loss: 46.0232 - val_loss: 43.1057
Epoch 874/10000
5/5 [==============================] - 0s 6ms/step - loss: 42.2381 -

5/5 [==============================] - 0s 5ms/step - loss: 65.2304 - val_loss: 139.2522
Epoch 945/10000
5/5 [==============================] - 0s 6ms/step - loss: 51.7384 - val_loss: 147.2963
Epoch 946/10000
5/5 [==============================] - 0s 7ms/step - loss: 55.9182 - val_loss: 93.0417
Epoch 947/10000
5/5 [==============================] - 0s 6ms/step - loss: 44.1821 - val_loss: 121.6218
Epoch 948/10000
5/5 [==============================] - 0s 6ms/step - loss: 47.5489 - val_loss: 75.9831
Epoch 949/10000
5/5 [==============================] - 0s 6ms/step - loss: 38.5954 - val_loss: 73.3816
Epoch 950/10000
5/5 [==============================] - 0s 5ms/step - loss: 39.7236 - val_loss: 67.0732
Epoch 951/10000
5/5 [==============================] - 0s 5ms/step - loss: 44.0029 - val_loss: 60.2665
Epoch 952/10000
5/5 [==============================] - 0s 5ms/step - loss: 66.4562 - val_loss: 82.9565
Epoch 953/10000
5/5 [==============================] - 0s 5ms/step - loss: 43.6285 - v

5/5 [==============================] - 0s 6ms/step - loss: 52.0367 - val_loss: 73.3556
Epoch 1024/10000
5/5 [==============================] - 0s 6ms/step - loss: 41.8896 - val_loss: 55.7418
Epoch 1025/10000
5/5 [==============================] - 0s 6ms/step - loss: 46.2683 - val_loss: 101.7289
Epoch 1026/10000
5/5 [==============================] - 0s 6ms/step - loss: 46.0965 - val_loss: 125.4318
Epoch 1027/10000
5/5 [==============================] - 0s 5ms/step - loss: 37.1626 - val_loss: 143.8006
Epoch 1028/10000
5/5 [==============================] - 0s 6ms/step - loss: 46.0165 - val_loss: 103.2048
Epoch 1029/10000
5/5 [==============================] - 0s 5ms/step - loss: 45.3684 - val_loss: 91.7062
Epoch 1030/10000
5/5 [==============================] - 0s 5ms/step - loss: 45.5848 - val_loss: 97.4706
Epoch 1031/10000
5/5 [==============================] - 0s 6ms/step - loss: 45.2733 - val_loss: 152.0606
Epoch 1032/10000
5/5 [==============================] - 0s 5ms/step - loss: 

5/5 [==============================] - 0s 5ms/step - loss: 58.0632 - val_loss: 80.8861
Epoch 1102/10000
5/5 [==============================] - 0s 5ms/step - loss: 45.0755 - val_loss: 83.2722
Epoch 1103/10000
5/5 [==============================] - 0s 5ms/step - loss: 43.5759 - val_loss: 49.8679
Epoch 1104/10000
5/5 [==============================] - 0s 5ms/step - loss: 48.6382 - val_loss: 51.0368
Epoch 1105/10000
5/5 [==============================] - 0s 5ms/step - loss: 47.8990 - val_loss: 59.9301
Epoch 1106/10000
5/5 [==============================] - 0s 5ms/step - loss: 42.1563 - val_loss: 82.5181
Epoch 1107/10000
5/5 [==============================] - 0s 6ms/step - loss: 53.9979 - val_loss: 177.0907
Epoch 1108/10000
5/5 [==============================] - 0s 6ms/step - loss: 72.3676 - val_loss: 185.0756
Epoch 1109/10000
5/5 [==============================] - 0s 5ms/step - loss: 54.6602 - val_loss: 175.8273
Epoch 1110/10000
5/5 [==============================] - 0s 6ms/step - loss: 48

5/5 [==============================] - 0s 6ms/step - loss: 45.3447 - val_loss: 88.5433
Epoch 1180/10000
5/5 [==============================] - 0s 6ms/step - loss: 39.6536 - val_loss: 52.7814
Epoch 1181/10000
5/5 [==============================] - 0s 6ms/step - loss: 40.8180 - val_loss: 128.5506
Epoch 1182/10000
5/5 [==============================] - 0s 6ms/step - loss: 42.9251 - val_loss: 132.0729
Epoch 1183/10000
5/5 [==============================] - 0s 6ms/step - loss: 41.1766 - val_loss: 115.7136
Epoch 1184/10000
5/5 [==============================] - 0s 7ms/step - loss: 49.8177 - val_loss: 118.3624
Epoch 1185/10000
5/5 [==============================] - 0s 6ms/step - loss: 37.1346 - val_loss: 95.3647
Epoch 1186/10000
5/5 [==============================] - 0s 8ms/step - loss: 32.3225 - val_loss: 47.9833
Epoch 1187/10000
5/5 [==============================] - 0s 6ms/step - loss: 38.7296 - val_loss: 122.7558
Epoch 1188/10000
5/5 [==============================] - 0s 6ms/step - loss: 

In [20]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input.iloc[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [21]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            test_input, 
                                            test_target, 
                                            train_split)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 18ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,74.155411,74.326218,74.668243,75.006668,79.795372,75.155807,1.72814,1.693926,1.727942,1.715376,1.328596,49.680771,1.338217,-0.627722,1.460718,1.723684,2.021902,-0.072618,-1.431263,-1.031935,-1.028112,-1.462158,-1.554167,-1.563202,-1.56782,-1.29146,1.705715,30.12989,76.895645,74.264107
Target,80.517,81.348,76.509,71.851,71.187,58.02,63.194,60.58,69.448,74.472,63.776,81.732,83.858,87.863,69.723,72.765,77.937,64.332,77.048,65.66,65.445,78.095,80.998,69.415,87.346,82.888,79.287,81.855,77.495,56.612
Error,6.361588,7.021782,1.840759,3.15567,8.608376,17.135807,61.465858,58.886078,67.720055,72.756622,62.447403,32.051231,82.519783,88.490723,68.262283,71.041313,75.915092,64.404617,78.479263,66.69194,66.473114,79.557159,82.55217,70.978203,88.913818,84.179459,77.581284,51.725113,0.599358,17.652107


In [22]:
display(mae)
display(mape)

53.848934

0.7304437

In [23]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [24]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             test_input, 
                                             test_target, 
                                             train_split)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 16ms/step
Ano-0: |Prediction[[510.98248]] - Target[852.6339999999999]| =  Error: [[341.6515]]; MAPE:[[0.40070122]]
1/1 [==============================] - 0s 15ms/step
Ano-0: |Prediction[[-2.226656]] - Target[893.1390000000001]| =  Error: [[895.36566]]; MAPE:[[1.0024931]]
1/1 [==============================] - 0s 16ms/step
Ano-5: |Prediction[[180.13608]] - Target[465.48300000000006]| =  Error: [[285.34692]]; MAPE:[[0.61301255]]


[array([[341.6515]], dtype=float32),
 array([[895.36566]], dtype=float32),
 array([[285.34692]], dtype=float32)]

507.45468

0.672069